# Import libraries

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import os

from PIL import Image

# Load data

https://www.kaggle.com/alxmamaev/flowers-recognition/version/2

In [ ]:
#classes = ['daisy','dandelion','rose','sunflower','tulip']
classes = ['daisy', 'tulip', 'rose']
num_classes = len(classes)

In [ ]:
labels = []
images = []

for flower in classes:
    for image in os.listdir('./Data/flowers/' + flower):
        images.append(image)
        labels.append(flower)

In [ ]:
print(len(labels))
print(len(images))

In [ ]:
idx = 500
image_file = './Data/flowers/' + labels[idx] + '/' + images[idx]
image = Image.open(image_file)
print(labels[idx])
image

# Data preprocessing / Feature engineering

### Convert images to numpy arrays

In [ ]:
y = np.asanyarray(labels)

In [ ]:
y

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
y

In [ ]:
y.shape

In [ ]:
def read_resize_image(idx):
    image_size = 64
    image_file = './Data/flowers/' + labels[idx] + '/' + images[idx]
    image = Image.open(image_file)
    image = image.resize((image_size, image_size))
    array = np.asarray(image, dtype="int32")
    array = np.expand_dims(array, axis=0)
    
    return array

In [ ]:
X = read_resize_image(0)

In [ ]:
X.shape

In [ ]:
#for idx in range(1, len(images)):
#    array = read_resize_image(idx)
#    X = np.concatenate((X, array), axis=0)

In [ ]:
#import pickle

#with open('./Data/flower_images.pkl','wb') as f:
#    pickle.dump(X, f)

Get the pickled data instead of creating it yourself:
https://www.dropbox.com/s/noxpojwwp1xoyjm/flower_images.pkl?dl=0

In [ ]:
import pickle
 
with open('./Data/flower_images.pkl','rb') as f:
     X = pickle.load(f)

In [ ]:
X.shape

# Split into train and test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

In [ ]:
y_train

# Build and train model

In [ ]:
import tensorflow
import keras
from keras import backend

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.applications import ResNet50

In [ ]:
# Clear tensorflow session
backend.clear_session()

In [ ]:
# Load the ResNet50 model
resnet_model = ResNet50(weights=None)

# Load the ResNet50 model with pretrained weights
#resnet_model = ResNet50(weights='imagenet')

In [ ]:
# resnet_model.summary()

In [ ]:
input_shape = (64,64,3)

In [ ]:
model = tensorflow.keras.Sequential()

model.add(InputLayer(input_shape=input_shape))
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

In [ ]:
# model.summary()

In [ ]:
# Define loss function
loss = 'sparse_categorical_crossentropy'

In [ ]:
# Define metrics
metrics = ['accuracy']

In [ ]:
# Define optimizer
optimizer = 'adam'

In [ ]:
# Compile model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [ ]:
# Train model
model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=1,
    validation_split=0.1,
    verbose=1
)

# Make predictions

In [ ]:
X_test[0]

In [ ]:
test_input = X_test[0]
test_input = np.expand_dims(test_input, axis=0)
print(test_input.shape)

In [ ]:
prediction = model.predict(test_input)
print(prediction)

In [ ]:
np.argmax(prediction)

In [ ]:
y_pred_prob = model.predict(X_test)

In [ ]:
y_pred_prob

In [ ]:
y_pred = np.argmax(y_pred_prob, axis=1)

# Performance metrics

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [ ]:
print(y_test[:12])
print(y_pred.squeeze()[:12])

In [ ]:
y_test = label_encoder.inverse_transform(y_test)
y_pred = label_encoder.inverse_transform(y_pred)

In [ ]:
print(y_test[:7])
print(y_pred.squeeze()[:7])

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.4}')

In [ ]:
precision = precision_score(y_test, y_pred, average=None)
print(precision)

In [ ]:
recall = recall_score(y_test, y_pred, average=None)
print(recall)

# Øvelser - Fine tune model

- Prøv at træne modellen med forskellige hyperparametre, og se hvordan det påvirker performance metrics.
- Prøv at tilføje flere forskellige lag til det neurale netværk. Se hvordan det påvirker performance metrics.
- Prøv at lade modellen træne over længere ved at sætte ```epochs``` op. Se hvordan det påvirker performance metrics.
- Test forskellige kombinationer af de overstående punkter, og se hvor god performance man kan få.

# Exercise - Fine tune model

- Try training the model with different hyperparameters, and see how it affects the performance metrics.
- Try adding more layers and different layers to the neural network. See how it affects the performance metrics.
- Try letting the model train for more time by increasing the number ```epochs```. See how it affects the performance metrics.
- Test different combinations of the methods in the previous bullets, and see how high performance you can get.